In [1]:
import nltk
sents = nltk.corpus.treebank_raw.sents()
print(sents)

[['.', 'START'], ['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov', '.', '29', '.'], ...]


In [2]:
tokens = []
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset-1)

In [3]:
def punct_features(tokens, i):
    return {'next-word-capitalized': tokens[i+1][0].isupper(),
            'prev-word': tokens[i-1].lower(),
            'punct': tokens[i],
            'prev-word-is-one-char': len(tokens[i-1]) == 1}

In [4]:
featuresets = [(punct_features(tokens, i), (i in boundaries)) for i in range(1, len(tokens)-1) if tokens[i] in '.?!']
print(featuresets)

[({'next-word-capitalized': False, 'prev-word': 'nov', 'punct': '.', 'prev-word-is-one-char': False}, False), ({'next-word-capitalized': True, 'prev-word': '29', 'punct': '.', 'prev-word-is-one-char': False}, True), ({'next-word-capitalized': True, 'prev-word': 'mr', 'punct': '.', 'prev-word-is-one-char': False}, False), ({'next-word-capitalized': True, 'prev-word': 'n', 'punct': '.', 'prev-word-is-one-char': True}, False), ({'next-word-capitalized': False, 'prev-word': 'group', 'punct': '.', 'prev-word-is-one-char': False}, True), ({'next-word-capitalized': True, 'prev-word': '.', 'punct': '.', 'prev-word-is-one-char': True}, False), ({'next-word-capitalized': False, 'prev-word': 'conglomerate', 'punct': '.', 'prev-word-is-one-char': False}, True), ({'next-word-capitalized': True, 'prev-word': '.', 'punct': '.', 'prev-word-is-one-char': True}, False), ({'next-word-capitalized': True, 'prev-word': 'reported', 'punct': '.', 'prev-word-is-one-char': False}, True), ({'next-word-capitalize

In [5]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.936026936026936

In [6]:
def segment_sentences(words):
    start = 0
    sents = []
    for i, word in list(enumerate(words))[:-1]:
        if word in '.?!' and classifier.classify(punct_features(words, i)) == True:
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        sents.append(words[start:])
    return sents

In [7]:
from nltk import word_tokenize
paragraph = "\"The forecast for tomorrow is deeply worrying,\" Senator Reynolds said. Areas across the country are expected to be at catastrophic fire danger levels and all Commonwealth agencies stand ready to assist state authorities and the community. \
Mr. Peter has a title Dr. but he is an Lt. Col. in army rank; served the nation for 20 years."
tokens = word_tokenize(paragraph)
#print(tokens)
sents = segment_sentences(tokens)
print(sents)
print("There are "+str(len(sents))+" sentences")
for sent in sents:
    print(" ".join(sent))

[['``', 'The', 'forecast', 'for', 'tomorrow', 'is', 'deeply', 'worrying', ',', "''", 'Senator', 'Reynolds', 'said', '.'], ['Areas', 'across', 'the', 'country', 'are', 'expected', 'to', 'be', 'at', 'catastrophic', 'fire', 'danger', 'levels', 'and', 'all', 'Commonwealth', 'agencies', 'stand', 'ready', 'to', 'assist', 'state', 'authorities', 'and', 'the', 'community', '.'], ['Mr.', 'Peter', 'has', 'a', 'title', 'Dr.', 'but', 'he', 'is', 'an', 'Lt.', 'Col.', 'in', 'army', 'rank', ';', 'served', 'the', 'nation', 'for', '20', 'years', '.']]
There are 3 sentences
`` The forecast for tomorrow is deeply worrying , '' Senator Reynolds said .
Areas across the country are expected to be at catastrophic fire danger levels and all Commonwealth agencies stand ready to assist state authorities and the community .
Mr. Peter has a title Dr. but he is an Lt. Col. in army rank ; served the nation for 20 years .
